In [ ]:
%load_ext autoreload
%autoreload

import matplotlib.pyplot as plt
import pandas as pd
from pipeline import *
from model import *
from bucket import *

figure = plt.figure(figsize=(20,11))

nblocks = 11
nticks = 10

pipeline = Pipeline(nblocks)
pipeline.run(nticks, nblocks)
to_plot = pipeline.measure(nticks)

df = pd.DataFrame(to_plot)
df.plot()